In [ ]:
# 1. Cài đặt các thư viện cần thiết
print("Bắt đầu cài đặt thư viện...")
%pip install ultralytics gradio -q
print("Cài đặt thành công!")

import gradio as gr
from ultralytics import YOLO
from PIL import Image
import os

# --- CẤU HÌNH CỦA BẠN (Đảm bảo chính xác) ---
# Tên file model YOLOv11 của bạn
model_path = 'best.pt'

# Ánh xạ ID của lớp (class_id) với thông tin sinh viên
# !!! ĐẢM BẢO THỨ TỰ NÀY KHỚP VỚI FILE data.yaml KHI BẠN TRAIN MODEL !!!
student_info = {
    0: "Vân Anh - MSSV:31241024702 - Quê quán: Kiên Giang - Sinh viên năm 2 ngành Công nghệ Logistics Đại học Kinh tế TP.HCM",
    1: "Đăng Khôi - MSSV:31241020719 - Quê quán: TP.HCM - Sinh viên năm 2 ngành Công nghệ Logistics Đại học Kinh tế TP.HCM",
    2: "Trung Hiếu MSSV:31241022719 - Quê quán: An Giang - Sinh viên năm 2 ngành Công nghệ Logistics Đại học Kinh tế TP.HCM"
}
# --- KẾT THÚC CẤU HÌNH CỦA BẠN ---

# 2. Kiểm tra và tải mô hình
print("Đang tải mô hình, vui lòng chờ...")
try:
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Không tìm thấy file '{model_path}'. Vui lòng tải file model của bạn lên Colab.")
    model = YOLO(model_path)
    print("Tải mô hình thành công!")
except Exception as e:
    print(f"Lỗi khi tải mô hình: {e}. Vui lòng kiểm tra lại đường dẫn hoặc file.")
    print("Sử dụng mô hình YOLOv8n mặc định để tiếp tục demo.")
    model = YOLO('yolov8n.pt')

# 3. Hàm xử lý và nhận diện
def recognize_face(input_image):
    if input_image is None:
        # Trả về placeholder trống và thông báo hướng dẫn
        return None, "<p style='text-align: center; color: #888;'>Vui lòng tải ảnh lên và nhấn nút 'Nhận Diện'</p>"

    image = Image.fromarray(input_image.astype('uint8'), 'RGB')
    results = model(image, conf=0.5)
    result = results[0]

    if len(result.boxes) == 0:
        # Trả về ảnh gốc và thông báo không tìm thấy
        return input_image, "<h3 style='color: #d9534f;'>Không tìm thấy khuôn mặt nào trong danh sách nhận diện.</h3>"

    annotated_image = result.plot()[..., ::-1]

    output_texts = []
    for box in result.boxes:
        class_id = int(box.cls)
        confidence = float(box.conf)
        label = student_info.get(class_id, f"Không rõ (ID: {class_id})")
        output_texts.append(f"<li><b>{label}</b> (Độ tin cậy: {confidence:.2f})</li>")

    final_text = "<h3>✨ Kết quả nhận diện:</h3><ul>" + "".join(output_texts) + "</ul>"

    return annotated_image, final_text

# 4. Xây dựng giao diện với Gradio
with gr.Blocks(theme=gr.themes.Soft(), title="FaceDetecting - Nhận Diện Thành Viên Nhóm") as demo:
    gr.Markdown(
        """
        <h1 style="text-align: center; color: #4CAF50;">✨ FaceDetecting ✨</h1>
        <p style="text-align: center; font-size: 1.1em; color: #555;">
            Hệ thống nhận diện thành viên nhóm của chúng tôi (Vân Anh, Đăng Khôi, Trung Hiếu)
            và hiển thị thông tin MSSV tương ứng.
        </p>
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(type="numpy", label="Ảnh Đầu Vào", height=350)
            btn = gr.Button("🚀 Nhận Diện Khuôn Mặt", variant="primary")

        with gr.Column(scale=1):
            gr.Markdown("<h3 style='text-align: center; color: #333;'>Kết Quả Nhận Diện</h3>")

            # === SỬA LỖI TẠI ĐÂY ===
            # Thêm 'interactive=False' để ô kết quả chỉ hiển thị, không cho phép tải lên.
            image_output = gr.Image(label="Ảnh đã được xử lý", interactive=False, height=350)
            text_output = gr.HTML(label="Thông tin chi tiết")

    # Liên kết nút bấm với hàm xử lý
    btn.click(fn=recognize_face, inputs=image_input, outputs=[image_output, text_output])

    gr.Markdown("<h3 style='text-align: center; color: #333; margin-top: 30px;'>Thử Với Các Ví Dụ Có Sẵn</h3>")
    gr.Examples(
        examples=[
            ["/content/Vanh.jpg"], # Thay bằng link ảnh thật
            ["/content/Khôi.jpg"], # Thay bằng link ảnh thật
            ["/content/Hiếu.jpg"]  # Thay bằng link ảnh thật
        ],
        inputs=image_input,
        outputs=[image_output, text_output],
        fn=recognize_face,
        cache_examples=False,
        label="Chọn một ảnh để xem kết quả nhanh"
    )

    gr.Markdown(
        """
        <p style="text-align: center; font-size: 0.8em; color: #999; margin-top: 50px;">
            © 2024 Nhóm XYZ - Powered by YOLOv11 & Gradio
        </p>
        """
    )

# 5. Khởi chạy ứng dụng
print("\nĐang khởi chạy giao diện, vui lòng chờ một lát...")
demo.launch(debug=True, share=True)

Bắt đầu cài đặt thư viện...
Cài đặt thành công!
Đang tải mô hình, vui lòng chờ...
Tải mô hình thành công!

Đang khởi chạy giao diện, vui lòng chờ một lát...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fdbfa8ddfb6ada6974.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-3479494814.py:42: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  image = Image.fromarray(input_image.astype('uint8'), 'RGB')


0: 640x480 1 Đăng Khôi 31241020719, 47.0ms
Speed: 3.8ms preprocess, 47.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 Đăng Khôi 31241020719, 8.2ms
Speed: 3.0ms preprocess, 8.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 480)


/tmp/ipython-input-3479494814.py:42: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  image = Image.fromarray(input_image.astype('uint8'), 'RGB')



0: 384x640 1 Đăng Khôi 31241020719, 50.2ms
Speed: 2.2ms preprocess, 50.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


/tmp/ipython-input-3479494814.py:42: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  image = Image.fromarray(input_image.astype('uint8'), 'RGB')
